<h1 style="font-family: Verdana">Goal</h1>
<p style="font-family: Verdana">I am implementing LDA and PCA to SVM</p>
<p style="font-family: Verdana"> SVM results contain overfitting to LDA, and underfitting to PCA </p>
<p style="font-family: Verdana">This notebook is my first implementation using that methods, i intend to improve and keep fixing or adding better results</p>

### Links
*  <a src="https://towardsdatascience.com/linear-discriminant-analysis-in-python-76b8b17817c2">LDA</a>              
*  <a src="https://www.kaggle.com/gauravsharma99/facial-emotion-recognition">References</a> 
*  <a src="https://www.kaggle.com/mouadriali/affectnetsample?rvi=1">DataSet of Images </a>

# Import
* Packages
* Definition of Emotions
* Define labels for each Emotion
* First Analyzes

In [ ]:
#from atom import ATOMClassifier, ATOMModel

import logging
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

# Import standard packages
from skopt.space.space import Integer, Categorical

# Keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

import os
import glob


pd.options.display.max_columns = None
sns.set(font_scale=1.4)
sns.set_style({'font.family': 'serif',
               'fontname': 'Times New Roman'})
mpl.rcParams['figure.dpi'] = 100

%matplotlib inline 

dicti = {0:'neut',1:'happ',2:'sad',3:'surp',4:'fear',5:'dist',6:'ang',7:'contp'}
rotulos = [dicti[0],dicti[1],dicti[2],dicti[3],dicti[4],dicti[5],dicti[6],dicti[7]]

In [ ]:
dt = pd.read_csv("../input/affectnetsample/train-sample-affectnet.csv",index_col=0)
dt.shape

In [ ]:
sns.countplot(dt['emotion'])

In [ ]:
dt.head()

# Data processing
* #### Choose Image Size
    * (64,64) or (32,32) , so just type 64 
* #### ETL of Images
    * Read Imagem
    * Reasize as 'tamanho'(size) choosed
    * Convert to Grayscale
    * Normalize each image
    * View Process performed

In [ ]:
tamanho = 64

### ETL de Imagens

In [ ]:
import time
from skimage import io
from sklearn import preprocessing
from skimage.transform import resize
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split

def preparing_image(address):
    retornar = preprocessing.normalize(resize(rgb2gray(io.imread(address)),(tamanho, tamanho)),norm='l2')
    retornar = resize(rgb2gray(io.imread(address)),(tamanho, tamanho))
    return retornar


def get_ETL_of_images(dt,tamanho):
    start_time = time.time()
    if ('data_64_X.npy' not in os.listdir()):
        
        name = 'data_'+str(tamanho)+'_' 
        print("...Generating databases...")
        
        y = None
        X = None
        
        # Seleciona imagens
        np.random.seed(0)
        rows_selected = np.random.randint(37553,size=2500)
        dt_selected = dt.iloc[rows_selected]

        for row in dt_selected.itertuples():
            y = [row.emotion] if y is None else np.concatenate( (y,[row.emotion]))
            x = preparing_image(row.image)
            X = [x] if X is None else np.concatenate( (X,[x]))

           
        with open(name+'X.npy', 'wb') as file:
            np.save(file,X)
        with open(name+'y.npy', 'wb') as file:
            np.save(file,y)
        
        all_time = time.time() - start_time
        if all_time > 60: print("{:.2f} minutes ".format(all_time/60))
        else: print("{:.2f} seconds ".format(all_time))
                  
        print(f'\n!! Base {name} completed !!')
        return (X,y)
    else:
        
        name = 'data_'+str(tamanho)
        print("...Reading databases...")
        
        print(name+'_X.npy')
        print(name+'_y.npy')
        if tamanho == 64:
            with open(name+'_X.npy', 'rb') as file:
                X = np.load(file) 
            with open(name+'_y.npy', 'rb') as file:
                y = np.load(file) 
            print("..Database X_34 Lido")
            
        elif tamanho == 32:
            with open(name+'X.npy', 'wb', 'rb') as file:
                X = np.load(file) 
            with open(name+'y.npy', 'rb') as file:
                y = np.load(file) 
            print("..Database X_32 Lido")
        #                      
        all_time = time.time() - start_time
        if all_time > 60: print("{:.2f} minutes ".format(all_time/60))
        else: print("{:.2f} seconds ".format(all_time))
        print(f'\n!! Base {name} completed !!')
        return (X,y)
        

In [ ]:
X , y = get_ETL_of_images(dt,tamanho)
print(f'Shape of X : {X.shape} \nShape of Y : {y.shape}')

In [ ]:
fig, ax = plt.subplots(3, 5,figsize=(8,7))

for i, axi in enumerate(ax.flat):
    axi.imshow(X[i], cmap='bone')
    axi.set(xticks=[], yticks=[], xlabel=dicti[y[i]-1])

# View Process performed

In [ ]:
dt_train = dt[0:50].copy()

imagem = io.imread(dt_train['image'][0])
#imagem_2 = io.imread(dt_test['image'][0])

fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(25,10))

print("****Imagem 1****")

before1 = imagem.copy()
ax[0].imshow(before1)
ax[0].set_title("Original")


grayscale1 = rgb2gray(imagem)
ax[1].imshow(grayscale1, cmap=plt.cm.gray)
ax[1].set_title("GrayScale")

rescaled1_64 = resize( grayscale1,(64,64) ) 
ax[2].imshow(rescaled1_64, cmap=plt.cm.gray)
ax[2].set_title("Rescaled(64x64)")

rescaled1_32 = resize( grayscale1,(32,32) ) 
ax[3].imshow(rescaled1_32, cmap=plt.cm.gray)
ax[3].set_title("Rescaled(32x32)")

# Pre-Processing
### PCA & SVM
* Parameters
* Grid Search
* SVM

### LDA & SVM
* Parameters
* SVM

### PCA & SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

In [ ]:
X_1D = X.reshape(X.shape[0],tamanho*tamanho)
print(f'Shape of X    : {X.shape} - Shape of Y : {y.shape}'+
     f'\nShape of X_1D : {X_1D.shape}   - Shape of Y : {y.shape}')

#### Parameters

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_rescaled = scaler.fit_transform(X_1D)
pca = PCA().fit(data_rescaled)

#-----------PLOT--------
plt.rcParams["figure.figsize"] = (8,6)
#
fig, ax = plt.subplots()
xi = np.arange(0, 2000, step=250)
y_pca = np.cumsum(pca.explained_variance_ratio_[[np.arange(0,2000,250)]])
#
plt.ylim(0.0,1.1)
plt.plot(xi, y_pca, marker='o', linestyle='--', color='b')
#
plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 2000, step=250)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')
#
plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)
#
ax.grid(axis='x')

plt.show()

In [ ]:
pca = PCA(n_components = 150, whiten='True', random_state=0)

svc = SVC(kernel='rbf')

model = make_pipeline(pca, svc)
X_pca = pca.fit_transform(X_1D)
X_pca.shape

In [ ]:
X_pca = X_pca.reshape(X_1D.shape[0], 15,10)

fig, ax = plt.subplots(3, 5,figsize=(8,7))
for i, axi in enumerate(ax.flat):
    axi.imshow(X_pca[i], cmap='bone')
    axi.set(xticks=[], yticks=[], xlabel=dicti[y[i]-1])

#### GridSearchCV

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X_1D, y, random_state=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

# fine tuning

param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}

# param_grid = {'svc__C': [9.8, 10, 10.2],
#               'svc__gamma': [0.0009, 0.001, 0.0011]}

grid = GridSearchCV(model, param_grid, cv=5)

grid.fit(Xtrain, ytrain)

print(grid.best_params_)

The best parameters

In [ ]:
model_best = grid.best_estimator_
print(model_best)

#### SVM
* Scores
* Confusion Matrix

In [ ]:
y_pred = model_best.predict(Xtest)

In [ ]:
fig, ax = plt.subplots(4, 6, figsize=(13,7))

for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(64, 64), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(dicti[y_pred[i]-1],
                   color='black' if y_pred[i] == ytest[i] else 'red')
    
fig.suptitle('Erros', size=24,color='red');

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(8,8))
ax.imshow(pca.mean_.reshape((tamanho,tamanho)), cmap="gray")
ax.set_xticks([])
ax.set_yticks([])
fig.suptitle('Average Face')

In [ ]:
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(ytest, y_pred)

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=rotulos,
            yticklabels=rotulos)
plt.xlabel('True label')
plt.ylabel('Predicted label');

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(ytest, y_pred, target_names=rotulos))

<a>https://www.mikulskibartosz.name/pca-how-to-choose-the-number-of-components/</a>

## LDA & SVM

In [ ]:
y.min()

In [ ]:
y_int = [ int(num)-1 for num in y ] 
y_lda = pd.Categorical.from_codes(y_int, rotulos)
y_lda

In [ ]:
dt_frame = pd.DataFrame(np.array(X_1D))
col_class = pd.DataFrame([dicti[int(num)-1] for num in y],columns=['class'])
dt_frame = pd.concat([dt_frame,col_class],axis=1)
#dt_frame.columns = [ i for i in range(len(dt_frame.columns))]
print(dt_frame.shape)
dt_frame.head()

In [ ]:
class_feature_means = pd.DataFrame(columns=rotulos)
for c, rows in dt_frame.groupby('class'):
#     print(rows.shape)
#     #print(rows.values)
#     print(rows.index[0])
#     print(rows.values[0][4096])
#     print("\n\n")

    class_feature_means[c] = rows.mean()

class_feature_means.head()

In [ ]:
within_class_scatter_matrix = np.zeros((4096 ,4096))
for c, rows in dt_frame.groupby('class'):
    rows = rows.drop(['class'], axis=1)
    
    s = np.zeros((4096 ,4096 ))
    for index, row in rows.iterrows():
            x, mc = row.values.reshape(4096,1), class_feature_means[c].values.reshape(4096,1)
            #print(x.shape,mc.shape)
            s += (x - mc).dot((x - mc).T)
           
    within_class_scatter_matrix += s

In [ ]:
feature_means = dt_frame.mean()
between_class_scatter_matrix = np.zeros((4096,4096))
for c in class_feature_means:    
    n = len(dt_frame.loc[dt_frame['class'] == c].index)
    mc, m = class_feature_means[c].values.reshape(4096,1), feature_means.values.reshape(4096,1)
    between_class_scatter_matrix += n * (mc - m).dot((mc - m).T)

In [ ]:
eigen_values, eigen_vectors = np.linalg.eig(np.linalg.inv(within_class_scatter_matrix).dot(between_class_scatter_matrix))

In [ ]:
pairs = [(np.abs(eigen_values[i]), eigen_vectors[:,i]) for i in range(len(eigen_values))]
pairs = sorted(pairs, key=lambda x: x[0], reverse=True)
for pair in pairs[:5]:
    print(pair[0])

In [ ]:
eigen_value_sums = sum(eigen_values)
print('Explained Variance')
for i, pair in enumerate(pairs[:5]):
    print('Eigenvector {}: {}'.format(i, (pair[0]/eigen_value_sums).real))

In [ ]:
w_matrix = np.hstack((pairs[0][1].reshape(4096,1), pairs[1][1].reshape(4096,1))).real

In [ ]:
X_lda = np.array(X_1D.dot(w_matrix))
X_lda.shape

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA 
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_lda = le.fit_transform(y)

In [ ]:
plt.xlabel('LD1')
plt.ylabel('LD2')
plt.grid(False)
plt.scatter(
    X_lda[:,0],
    X_lda[:,1],
    c=y,
    cmap='rainbow',
    alpha=0.7,
    edgecolors='b'
)

In [ ]:
lda = LDA()
X_lda = lda.fit_transform(X_1D, y)
lda.explained_variance_ratio_

### Parameters

In [ ]:
plt.xlabel('LD1')
plt.ylabel('LD2')
plt.grid(False)
plt.scatter(X_lda[:,0],
    X_lda[:,1],
    c=y,
    cmap='rainbow',
    alpha=0.7,
    edgecolors='b'
)

#### SVM

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X_lda, y, random_state=0)

In [ ]:
le = LabelEncoder()
y_le = le.fit_transform(y)
#
lda = LDA()
# X_train = lda.fit_transform(X_train, y_train)
# X_test = lda.transform(X_test)
model2 = make_pipeline(lda, svc)
X_lda = lda.fit_transform(X_1D,y_le)

In [ ]:
lda.explained_variance_ratio_

In [ ]:
grid = GridSearchCV(model2, param_grid, cv=5)

grid.fit(Xtrain, ytrain)

print(grid.best_params_)

In [ ]:
model_best = grid.best_estimator_
print(model_best)

In [ ]:
y_pred = model_best.predict(Xtest)

In [ ]:
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(ytest, y_pred)

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=rotulos,
            yticklabels=rotulos)
plt.xlabel('True label')
plt.ylabel('Predicted label');

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(ytest, y_pred, target_names=rotulos))

## Future Improvements
* ### View images by delimiting face

In [ ]:
from skimage.feature import Cascade
from skimage import data
from matplotlib import patches

# Load the trained file from the module root.
trained_file = data.lbp_frontal_face_cascade_filename()

# Initialize the detector cascade.
detector = Cascade(trained_file)

img = rescaled1_64.copy()


detected = detector.detect_multi_scale(img=img,
                                       scale_factor=1.2,
                                       step_ratio=1,
                                       min_size=(4, 4),
                                       max_size=(64, 64))
plt.imshow(img)
img_desc = plt.gca()
plt.set_cmap('gray')

for patch in detected:

    img_desc.add_patch(
        patches.Rectangle(
            (patch['c'], patch['r']),
            patch['width'],
            patch['height'],
            fill=False,
            color='r',
            linewidth=1.5
        )
    )  
plt.grid(False)
plt.show()

In [ ]:
cropped = img[detected[0]['c']:detected[0]['width'],detected[0]['r']:detected[0]['height']]
plt.grid(False)
plt.imshow(cropped)
print(cropped.shape)    

* Scroll through all images
     * Extract face
     * Generate new column
     * Calculate the smallest and largest shape
         * Set a fixed cut size for everyone